In [1]:
from dimod import ConstrainedQuadraticModel, Binary, Integer, SampleSet, BinaryArray
from dwave.system import LeapHybridCQMSampler

In [2]:
tasks = [f"task_{i}" for i in range(12)]
processors = [f"processor_{i}" for i in range(4)]

In [3]:
new_assignment = [[BinaryArray([f"x{4*i + j}" for j in range(len(processors))])] for i in range(len(tasks))]

In [41]:
# new_assignment

In [5]:
new_assignment[11][0][0]

BinaryQuadraticModel({'x44': 1.0}, {}, 0.0, 'BINARY')

In [6]:
cqm = ConstrainedQuadraticModel()

In [7]:
original_time = [
    100, 100, 100,  # processor 0 
    200, 200, 200,  # processor 1 
    75, 75, 75,     # processor 2 
    150, 150, 150,  # processor 3 
]

In [8]:
original_assignment = [
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    
    [0, 1, 0, 0],
    [0, 1, 0, 0],
    [0, 1, 0, 0],
    
    [0, 0, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 1, 0],
    
    [0, 0, 0, 1],
    [0, 0, 0, 1],
    [0, 0, 0, 1],
]

In [44]:
# processor_speeds = [2.0, 1.0, 4.0, 1.0]
processor_speeds = [1.0, 1.0, 1.0, 1.0]

delay_for_moving_task = 0

In [43]:
penalties_for_moving_task = []

for task_id, task_name in enumerate(tasks):
    original_processor_id = 0
    for processor_id, processor_name in enumerate(processor_speeds):
        if original_assignment[task_id][processor_id] == 1:
            original_processor_id = processor_id    
    
    penalty_for_change_of_speed_and_delay = []
    for processor_id, new_processor_speed in enumerate(processor_speeds):

        time_in_new_speed = processor_speeds[original_processor_id]/new_processor_speed * original_time[task_id]
        penalty = time_in_new_speed - original_time[task_id]
        
        
        if processor_id != original_processor_id:
            penalty += delay_for_moving_task
            
        penalty_for_change_of_speed_and_delay.append(penalty)

    penalties_for_moving_task.append(penalty_for_change_of_speed_and_delay)       

###### print stuff
for row in penalties_for_moving_task:
    print(row)

[0.0, 130.0, -20.0, 130.0]
[0.0, 130.0, -20.0, 130.0]
[0.0, 130.0, -20.0, 130.0]
[-70.0, 0.0, -120.0, 30.0]
[-70.0, 0.0, -120.0, 30.0]
[-70.0, 0.0, -120.0, 30.0]
[105.0, 255.0, 0.0, 255.0]
[105.0, 255.0, 0.0, 255.0]
[105.0, 255.0, 0.0, 255.0]
[-45.0, 30.0, -82.5, 0.0]
[-45.0, 30.0, -82.5, 0.0]
[-45.0, 30.0, -82.5, 0.0]


In [11]:
avg = sum(original_time)/4 # 393.75
obj_function = 0

for processor_id, processor_name in enumerate(processors):
    
    processor_time = 0
    for task_id, task_name in enumerate(tasks):
        print(task_id, processor_id)
        processor_time += (
                        new_assignment[task_id][0][processor_id] * 
                         (original_time[task_id] + penalties_for_moving_task[task_id][processor_id])
                        )
        
    print(f"Processor {processor_name} time {processor_time}")
    obj_function += (processor_time - avg)**2
    
print(f"Objective function {obj_function/len(processors)}")

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
Processor processor_0 time BinaryQuadraticModel({'x0': 100.0, 'x4': 100.0, 'x8': 100.0, 'x12': 200.0, 'x16': 200.0, 'x20': 200.0, 'x24': 75.0, 'x28': 75.0, 'x32': 75.0, 'x36': 150.0, 'x40': 150.0, 'x44': 150.0}, {}, 0.0, 'BINARY')
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
Processor processor_1 time BinaryQuadraticModel({'x1': 100.0, 'x5': 100.0, 'x9': 100.0, 'x13': 200.0, 'x17': 200.0, 'x21': 200.0, 'x25': 75.0, 'x29': 75.0, 'x33': 75.0, 'x37': 150.0, 'x41': 150.0, 'x45': 150.0}, {}, 0.0, 'BINARY')
0 2
1 2
2 2
3 2
4 2
5 2
6 2
7 2
8 2
9 2
10 2
11 2
Processor processor_2 time BinaryQuadraticModel({'x2': 100.0, 'x6': 100.0, 'x10': 100.0, 'x14': 200.0, 'x18': 200.0, 'x22': 200.0, 'x26': 75.0, 'x30': 75.0, 'x34': 75.0, 'x38': 150.0, 'x42': 150.0, 'x46': 150.0}, {}, 0.0, 'BINARY')
0 3
1 3
2 3
3 3
4 3
5 3
6 3
7 3
8 3
9 3
10 3
11 3
Processor processor_3 time BinaryQuadraticModel({'x3': 100.0, 'x7': 100.0, 'x11': 100.0, 'x15': 200.0, 'x1

In [18]:
new_assignment[task_id][0][3] 

BinaryQuadraticModel({'x47': 1.0}, {}, 0.0, 'BINARY')

In [23]:
for task_id, task in enumerate(tasks):
    cqm.add_constraint(new_assignment[task_id][0][0]
                       + new_assignment[task_id][0][1] 
                       + new_assignment[task_id][0][2] 
                       + new_assignment[task_id][0][3] == 1)

In [24]:
cqm.set_objective(obj_function/len(processors))

In [25]:
sampler = LeapHybridCQMSampler()
raw_sampleset = sampler.sample_cqm(cqm, time_limit=5)

In [26]:
feasible_sampleset = raw_sampleset.filter(lambda d: d.is_feasible)

In [31]:
feasible_sampleset

SampleSet(rec.array([([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0.],   117.1875, 1, [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],  True),
           ([0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],   117.1875, 1, [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],  True),
           ([0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.],   117.1875, 1, [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],  True),
           ([0., 1., 0., 1., 0., 1., 0., 0., 

In [29]:
amount_of_moved_tasks = 0

for task_id, task_name in enumerate(tasks):
    check_if_moved = 1
    for processor_id, processor_name in enumerate(processors):
        check_if_moved -= (original_assignment[task_id][processor_id]
                           * new_assignment[task_id][0][processor_id])
        
    
    if check_if_moved == 1:
        print(f"Task {task_id} migrated")

    amount_of_moved_tasks += check_if_moved
print(f"\nAmount of moved tasks: {amount_of_moved_tasks}")


Amount of moved tasks: BinaryQuadraticModel({'x0': -1.0, 'x1': -0.0, 'x2': -0.0, 'x3': -0.0, 'x4': -1.0, 'x5': 0.0, 'x6': 0.0, 'x7': 0.0, 'x8': -1.0, 'x9': 0.0, 'x10': 0.0, 'x11': 0.0, 'x12': 0.0, 'x13': -1.0, 'x14': 0.0, 'x15': 0.0, 'x16': 0.0, 'x17': -1.0, 'x18': 0.0, 'x19': 0.0, 'x20': 0.0, 'x21': -1.0, 'x22': 0.0, 'x23': 0.0, 'x24': 0.0, 'x25': 0.0, 'x26': -1.0, 'x27': 0.0, 'x28': 0.0, 'x29': 0.0, 'x30': -1.0, 'x31': 0.0, 'x32': 0.0, 'x33': 0.0, 'x34': -1.0, 'x35': 0.0, 'x36': 0.0, 'x37': 0.0, 'x38': 0.0, 'x39': -1.0, 'x40': 0.0, 'x41': 0.0, 'x42': 0.0, 'x43': -1.0, 'x44': 0.0, 'x45': 0.0, 'x46': 0.0, 'x47': -1.0}, {}, 12.0, 'BINARY')


In [37]:
# for sample in feasible_sampleset.samples()[0]:   
#     print(sample)

In [45]:
chosen_sample

{'x0': 0.0, 'x1': 0.0, 'x10': 0.0, 'x11': 0.0, 'x12': 0.0, 'x13': 1.0, 'x14': 0.0, 'x15': 0.0, 'x16': 0.0, 'x17': 0.0, 'x18': 1.0, 'x19': 0.0, 'x2': 1.0, 'x20': 0.0, 'x21': 1.0, 'x22': 0.0, 'x23': 0.0, 'x24': 1.0, 'x25': 0.0, 'x26': 0.0, 'x27': 0.0, 'x28': 1.0, 'x29': 0.0, 'x3': 0.0, 'x30': 0.0, 'x31': 0.0, 'x32': 0.0, 'x33': 0.0, 'x34': 1.0, 'x35': 0.0, 'x36': 1.0, 'x37': 0.0, 'x38': 0.0, 'x39': 0.0, 'x4': 0.0, 'x40': 0.0, 'x41': 0.0, 'x42': 0.0, 'x43': 1.0, 'x44': 0.0, 'x45': 0.0, 'x46': 0.0, 'x47': 1.0, 'x5': 0.0, 'x6': 0.0, 'x7': 1.0, 'x8': 1.0, 'x9': 0.0}

In [38]:
sample_id = 0
chosen_sample = feasible_sampleset.samples()[sample_id]

for task_id, task in enumerate(tasks):
    print(chosen_sample[f'x{4*task_id}'],
          chosen_sample[f'x{4*task_id + 1}'],
          chosen_sample[f'x{4*task_id + 2}'],
          chosen_sample[f'x{4*task_id + 3}'],
         )

0.0 0.0 1.0 0.0
0.0 0.0 0.0 1.0
1.0 0.0 0.0 0.0
0.0 1.0 0.0 0.0
0.0 0.0 1.0 0.0
0.0 1.0 0.0 0.0
1.0 0.0 0.0 0.0
1.0 0.0 0.0 0.0
0.0 0.0 1.0 0.0
1.0 0.0 0.0 0.0
0.0 0.0 0.0 1.0
0.0 0.0 0.0 1.0
